Pandas'ın başlıca özelliklerinden biri de, kolayca elemanlarında işlem yapılmasına izin vermesidir. Bu işlevselliğin bir çoğu Numpy ve [ufuncs](#)'dan gelmektedir.

Pandas Numpy ile çalışmak için tasarlandığından dolayı bütün Numpy evrensel fonksiyonları Pandas'ta da çalışır.

## Ufuncs: Index'lerin Korunumu

Pandas nesnelerinde Numpy ufunc'larını kullanabilirsiniz. Ve bu işlemler sırasında Index'ler olduğu gibi kalır, zarar görmez.

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({"A":[1, 2, 3], "B":[10, 20, 30]})
df

,A,B
0,1,10
1,2,20
2,3,30


In [3]:
np.multiply(df, 2) # 2 ile çarp

,A,B
0,2,20
1,4,40
2,6,60


## Ufuncs: Aynı Olmayan Index'ler

İki serie veya Dataframe arasında aritmetik bir işlem yaparsanız, Pandas işlem sırasında indexleri birleştirecektir. Bu özellik, uygun bir veri üzerinde çalışıyorsanız çok kullanışlıdır. Bu bölümde indexlerin aynı olmaması durumunu inceleyeceğiz.

### Serie'lerde Aynı Olmayan İndexler

`nufus` değişkenimizde üç şehrin nüfusunu, `alan` değişkeninde de üç şehrin yüzey alanını tanımlayalım. Yalnız, sonuncu şehirler aynı olmasın.

In [4]:
nufus = pd.Series({'Ankara': 5639076, 'İzmir': 4367251, 'İstanbul': 15519267}) # 2020
alan =  pd.Series({'Ankara': 25632,   'İzmir': 11891,   'Burdur':   7175}) # km2
nufus / alan

Ankara      220.001404
Burdur             NaN
İstanbul           NaN
İzmir       367.273652
dtype: float64

Gördüğünüz üzere, Serie'lerden birinde olup diğerinde olmayan index'ler için işlem yapamadı ve NaN değeri döndürdü. NaN (Not a numeric), sayı olmayan değer demektir ve kayıp veriyi temsil eder. Kayıp veriyi işlemeyi [bir sonraki bölümde](#) göreceğiz.

Python'da aritmetik işlemlerde kayıp veriler için ön tanımı olarak NaN kullanılır. Eğer kayıp veriler için NaN yerine başka bir şey kullanmak isterseniz `fill_value` parametresini kullanabilirsiniz. (Not: `np.divide`ın `fill_value` parametresi yoktur)

In [5]:
nufus.divide(alan, fill_value=0)

Ankara      220.001404
Burdur        0.000000
İstanbul           inf
İzmir       367.273652
dtype: float64

Burada dikkat etmeniz gereken şey, `fill_value` NaN değerleri değil kayıp verileri doldurur ve işlemleri öyle yapar. Bu yüzden NaN'ların yerinde 0 ve inf (infinity) yazıyor.

### DataFrame'lerde Aynı Olmayan İndexler

In [6]:
x = pd.DataFrame([[1, 2], [9, 8]], columns=['A', 'B'])
x

,A,B
0,1,2
1,9,8


In [7]:
y = pd.DataFrame([[10, 20, 30], [90, 80, 70]], columns=['B', 'A', 'C'])
y

,B,A,C
0,10,20,30
1,90,80,70


In [8]:
x + y

,A,B,C
0,21,12,NaN
1,89,98,NaN


Gördüğünüz üzere işlemi yaptıktan sonra indexleri sıralayıp gösterdi ve olmayan verilerin yerine NaN koydu.

DataFrame'lerde de aynı şekilde `fill_value` kullanabiliriz. Aşağıdaki örnekte kayıp verilerin yerine, x dizisinin değerlerinin ortalamasını yazıcaz.

In [9]:
ortalama = x.stack().mean() # A'nın tüm değerlerini tek boyutlu bir dizi yapıp değerlerin ortalamasını al
ortalama

5.0

In [10]:
x.add(y, fill_value=ortalama)

,A,B,C
0,21,12,35.0
1,89,98,75.0


Pandas fonksiyonları aşağıda verilmiştir.

| Operatör | Fonksiyon                  |
|----------|----------------------------|
| +        | add()                      |
| -        | sub(), subtract()          |
| *        | mul(), multiply()          |
| /        | div(), divide(), truediv() |
| //       | floordiv()                 |
| %        | mod()                      |
| **       | pow()                      |

## Ufuncs: DataFrame ve Serie'ler Arasında İşlem Yapma

DataFrame ile Serie arasındaki işlemler, iki boyutlu ile bir boyutlu diziler arasındaki işlemlere benzer.

In [11]:
a = np.arange(1,10).reshape(3,3)
a

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [12]:
a - np.array([4, 5, 6])

array([[-3, -3, -3],
       [ 0,  0,  0],
       [ 3,  3,  3]])

Numpy'ın Boardcasting kurallarında olduğu gibi (Hatırlamıyorsanız: [Numpy Boardcasting](#) bölümünde görmüştük.) ikinci dizi birinci diziye uyacak şekilde çoğaltılıp çıkartma işlemi yapıldı. Yani ikinci dizinin satır sayısı çoğaltıldı.

Pandas dizilerinde de işlemler benzer şekilde gerçekleşir.

In [13]:
y = pd.DataFrame([[10, 20, 30], [90, 80, 70]], columns=['A', 'B', 'C'])
y

,A,B,C
0,10,20,30
1,90,80,70


In [14]:
y - pd.Series([4, 5, 6], index=("A", "B", "C"))

,A,B,C
0,6,15,24
1,86,75,64


Eğer ikinci dizinin diğerine uyacak şekilde çoğaltılması işlemini satır değilde sütun bazlı yapmak isterseniz bunu axis parametresinde belirtebilirsiniz.

In [15]:
y.subtract(pd.Series([4, 5]), axis=0)

,A,B,C
0,6,16,26
1,85,75,65
